This notebook is going to contain the dataset, and model(s) which is going to be used to predict the open and low for the next day. So far, a dataset has been created, and various techincal indicators for bitcoin price movement have been added into it, the aggregated sentiment scores, generated using FinBERT are also the feature in my final dataset. The main focus of this notebook would be to predict open and low for the next day without any further normalization or preprocessing

In [1]:
### Importing the necessary libraries
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('data/final_data.csv', index_col='Date')
data.head(3)

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,...,MACD,Signal Line,Bollinger_SMA,Upper_Band_BB,Lower_Band_BB,ATR,%K,%D,OBV,aggregated_sentiment
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-08,908.174988,942.723999,887.249023,911.198975,911.198975,158715008,993.669277,976.372637,958.356795,954.544005,...,39.007756,51.508799,944.295145,1103.190195,785.400095,69.768864,23.845569,19.694043,1865537512,0.000000
2017-01-10,902.440002,914.872986,901.059998,907.679016,907.679016,115808000,957.228856,974.208282,935.275702,942.319269,...,25.901844,43.209113,953.062646,1089.773628,816.351665,69.404432,22.887871,22.767155,1839468520,0.421053
2017-01-11,908.114990,919.447998,762.765015,777.757019,777.757019,310928992,903.375575,960.053711,895.896031,920.377635,...,10.743321,36.715954,948.723499,1101.794673,795.652324,77.544429,3.500066,15.985321,1528539528,0.349206


In order to get our target variables, I can simply shift the high and low by -1, this would provide me the target columns, as the open and low for the next day.


In [4]:
data['y_high'] = data['High'].shift(-1)
data['y_low'] = data['Low'].shift(-1)

In [5]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,...,Bollinger_SMA,Upper_Band_BB,Lower_Band_BB,ATR,%K,%D,OBV,aggregated_sentiment,y_high,y_low
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-08,908.174988,942.723999,887.249023,911.198975,911.198975,158715008,993.669277,976.372637,958.356795,954.544005,...,944.295145,1103.190195,785.400095,69.768864,23.845569,19.694043,1865537512,0.000000,914.872986,901.059998
2017-01-10,902.440002,914.872986,901.059998,907.679016,907.679016,115808000,957.228856,974.208282,935.275702,942.319269,...,953.062646,1089.773628,816.351665,69.404432,22.887871,22.767155,1839468520,0.421053,919.447998,762.765015
2017-01-11,908.114990,919.447998,762.765015,777.757019,777.757019,310928992,903.375575,960.053711,895.896031,920.377635,...,948.723499,1101.794673,795.652324,77.544429,3.500066,15.985321,1528539528,0.349206,826.245972,755.755981


In [7]:
data.tail(3) ### Last row has target variables as Nan, I will simply drop it

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,...,Bollinger_SMA,Upper_Band_BB,Lower_Band_BB,ATR,%K,%D,OBV,aggregated_sentiment,y_high,y_low
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-25,60266.281250,62258.261719,60239.750000,61804.640625,61804.640625,29201215431,63342.940290,64919.310826,63102.154092,64532.192945,...,66238.408984,71772.224356,60704.593613,1889.449498,28.154778,14.294931,1457322360739,0.650794,62434.136719,60695.187500
2024-06-26,61789.675781,62434.136719,60695.187500,60811.277344,60811.277344,22506003064,62750.223214,64388.602958,62529.434905,64036.070865,...,65741.114648,71352.163184,60130.066113,1793.986049,22.629655,21.838150,1434816357675,0.360000,62293.863281,60585.332031
2024-06-27,60811.226562,62293.863281,60585.332031,61604.800781,61604.800781,21231745045,62289.672433,64020.631696,62298.276374,63711.901520,...,65354.225391,70986.709987,59721.740795,1768.794085,34.546406,28.443613,1456048102720,0.478261,NaN,NaN


In [11]:
data.drop(index='2024-06-27' , inplace=True, axis=0)
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,...,Bollinger_SMA,Upper_Band_BB,Lower_Band_BB,ATR,%K,%D,OBV,aggregated_sentiment,y_high,y_low
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-24,63173.351562,63292.527344,58601.699219,60277.414062,60277.414062,43152133651,63819.526786,65314.124442,63534.658581,64951.816378,...,66702.317969,72226.724543,61177.911394,1989.970982,14.730016,9.213542,1428121145308,0.305556,62258.261719,60239.750000
2024-06-25,60266.281250,62258.261719,60239.750000,61804.640625,61804.640625,29201215431,63342.940290,64919.310826,63102.154092,64532.192945,...,66238.408984,71772.224356,60704.593613,1889.449498,28.154778,14.294931,1457322360739,0.650794,62434.136719,60695.187500
2024-06-26,61789.675781,62434.136719,60695.187500,60811.277344,60811.277344,22506003064,62750.223214,64388.602958,62529.434905,64036.070865,...,65741.114648,71352.163184,60130.066113,1793.986049,22.629655,21.838150,1434816357675,0.360000,62293.863281,60585.332031
